In [1]:
import sys
sys.path.append('../src')
from dataloader import create_dataloaders
# from dataloader_ER import create_dataloaders
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_vmap import LORAEngineDebertaMultiClass

# from influence import IFEngine
from influence_hyperinf import IFEngine

import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
model_name_or_path="../../model/deberta-v3-base"
# model_name_or_path="../../model/roberta-large"
task="mrpc"
# task = "qnli"
noise_ratio=0
batch_size=32
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = ['value']
target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:0"
num_epochs=3
lr=3e-3

In [ ]:
### BERT
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
# model_name_or_path="../../model/deberta-v3-base"
model_name_or_path = '/data/home/wangys/model/bert-base-uncased/'
# model_name_or_path="../../model/roberta-large"
task="mrpc"
# task = "qnli"
noise_ratio=0
batch_size=32
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
target_modules = ['value']
# target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:0"
num_epochs=3
lr=3e-3

In [3]:
# fine-tuning models
# dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
#                                            task=task,
#                                            noise_ratio=noise_ratio,
#                                            batch_size=batch_size,
#                                            train_file = '../ER/semi-text-c-FUSER/train.json',
#                                            valid_file = '../ER/semi-text-c-FUSER/valid.json',
#                                            test_file = '../ER/semi-text-c-FUSER/test.json')


dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                        #    train_file = '../ER/semi-text-c/train.json',
                                        #    valid_file = '../ER/semi-text-c/valid.json',
                                        #    test_file = '../ER/semi-text-c/test.json',
                                           max_length=256)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/mrpc',
                            valid_each_epoch=True,
                            cal_grad_per_sample = True)

In [5]:
for step,batch in enumerate(train_dataloader):
    print(batch['id'])
    break
    # print(batch['input_ids'][0])
    

tensor([2451, 2325, 3258,  243, 3213, 2449, 1249, 3573, 1120, 2242, 1643,  370,
        1781,  229, 1843, 1983, 2358, 2732, 3303, 2340,   96, 1886, 1372, 2255,
        3243,  217, 2429, 1166, 2277, 1968,  729,  453])


In [ ]:
# 
lora_engine.eval_LORA_model()

In [4]:
lora_engine.build_LORA_model()
tr_grad_dict,val_grad_dict = lora_engine.train_LORA_model()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ../../model/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 184,720,132 || trainable%: 0.1605
Total Steps:345


100%|██████████| 115/115 [00:05<00:00, 19.78it/s]


Epoch 1: Training Loss = 0.5401092377693757


100%|██████████| 13/13 [00:00<00:00, 22.74it/s]


...

Epoch 3: Training Loss = 0.20453087376511614


100%|██████████| 13/13 [00:00<00:00, 22.88it/s]


Epoch 3: {'f1': 0.9200710479573713}


100%|██████████| 54/54 [00:02<00:00, 22.78it/s]

Prediction Result on Test Data: {'f1': 0.9184805804524114}


In [11]:
import torch
torch.save(tr_grad_dict,'../output/tr_grad_dict_all.pkl')
torch.save(val_grad_dict,'../output/val_grad_dict_all.pkl')

In [ ]:
lora_engine.save('../models/mrpc-all')
# lora_engine.load('../models/qnli')

In [18]:
tr_grad_dict[0][0]['base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_B.default.weight'].dtype

torch.float32

In [12]:
lora_engine.save_lora('../models/mrpc-deberta-lora')


[2024-12-17 03:14:17,312] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/wangys/anaconda3/envs/deepspeed/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


In [ ]:
lora_engine.load_pretrained_network('../models/mrpc-deberta-lora')
lora_engine.eval_LORA_model()

In [ ]:
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient_iterative(tokenized_datasets, collate_fn, batch_size=32)

In [ ]:
import torch
tr_grad_dict = torch.load('../output/tr_grad_dict_all.pkl')
val_grad_dict = torch.load('../output/val_grad_dict_all.pkl')

In [32]:
from time import time
from tqdm.notebook import tqdm
import sys
from collections import defaultdict
import pandas as pd
import pickle, os
import torch
import numpy as np
def compute_hvp_and_IF(tr_grad_dict, val_grad_avg_dict, weight_name, device='cuda:3', lambda_const_param=10, n_iteration=30):
    '''
    Compute the influence function score by our method HyperINF
    '''

    def schulz_inverse_stable(A, damping_factor=0, max_iterations=20, tol=1e-6):
        n = A.shape[0]
        I = torch.eye(n, device=A.device)
        A_damped = A + damping_factor * I  # Apply damping for stability
        X = torch.eye(n, device=A.device) * 0.00005  # Initial estimate of inverse matrix

        for _ in range(max_iterations):
            X = X @ (2 * I - A_damped @ X)

        return X

    start_time = time()
    hvp_iterative_dict = {}
    if_tmp_dict = {}

    # 将相关变量放到指定设备上
    # tr_grad_dict_layer = {tr_id: tr_grad_dict[tr_id][weight_name].to(device) for tr_id in tr_grad_dict}
    # val_grad_avg_dict_layer = val_grad_avg_dict[weight_name].to(device)
    
    tr_grad_dict_layer = {tr_id: tr_grad_dict[tr_id][weight_name] for tr_id in tr_grad_dict}
    val_grad_avg_dict_layer = val_grad_avg_dict[weight_name]
    # print(val_grad_avg_dict_layer['base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_B.default.weight'].device)

    for tr_id in tr_grad_dict:
        if_tmp_value = 0

        # lambda_const computation
        S = torch.zeros(len(tr_grad_dict.keys())).to(device)
        for tr_id_inner in tr_grad_dict:
            tmp_grad = tr_grad_dict_layer[tr_id_inner]
            S[tr_id_inner] = torch.mean(tmp_grad**2)
        lambda_const = torch.mean(S) / lambda_const_param  # layer-wise lambda

        # iterative hvp computation
        G_l = torch.zeros((tr_grad_dict_layer[0].T @ tr_grad_dict_layer[0]).shape).to(device)
        for tr_id_inner in tr_grad_dict:
            tmp_grad = tr_grad_dict_layer[tr_id_inner]
            G_l += tmp_grad.T @ tmp_grad / len(tr_grad_dict)
        
        G_l = G_l + lambda_const * torch.eye(G_l.shape[0], device=G_l.device)
        # G_l_inv = torch.inverse(G_l)
        G_l_inv = schulz_inverse_stable(G_l, damping_factor=0.001, max_iterations=n_iteration, tol=1e-6)
        # print(val_grad_avg_dict_layer.device,G_l_inv.device)
        hvp_iterative_dict[weight_name] = torch.tensor(val_grad_avg_dict_layer @ G_l_inv)

        if_tmp_value += torch.sum(hvp_iterative_dict[weight_name] * tr_grad_dict_layer[tr_id])
        if_tmp_dict[tr_id] = -if_tmp_value.cpu()

    # 从设备中删除变量
    del tr_grad_dict_layer
    del val_grad_avg_dict_layer

    IF_dict = pd.Series(if_tmp_dict, dtype=float).to_numpy()
    time_taken = time() - start_time

    print("Time taken for HyperINF: ", time_taken)
    return hvp_iterative_dict, IF_dict


In [3]:
def compute_val_grad_avg(val_grad_dict):
    # Compute the avg gradient on the validation dataset
    n_val = len(val_grad_dict)
    val_grad_avg_dict={}
    for weight_name in val_grad_dict[0]:
        if weight_name.__contains__('base_model'):
            val_grad_avg_dict[weight_name]=torch.zeros(val_grad_dict[0][weight_name].shape)
            for val_id in val_grad_dict:
                val_grad_avg_dict[weight_name] += val_grad_dict[val_id][weight_name] / n_val
        # else:
        #     val_grad_avg_dict[weight_name] = val_grad_dict[weight_name]
    # val_grad_avg_dict['ids'] = val_grad_dict['ids']
    return val_grad_avg_dict

In [4]:
val_grad_avg_dict = {}
for epoch in val_grad_dict.keys():
    val_grad_avg_dict[epoch] = compute_val_grad_avg(val_grad_dict[epoch])

In [33]:
IF_device = 'cuda:3' ## 避免爆显存
# tr_grad_dict_IF = tr_grad_dict.copy()
# val_grad_dict_IF = val_grad_dict.copy()
# ## to cuda
for epoch in tr_grad_dict.keys():
    for key in tqdm(tr_grad_dict[epoch]): ## 尝试转为fp16计算
        for kk in tr_grad_dict[epoch][key]:
            tr_grad_dict[epoch][key][kk] = tr_grad_dict[epoch][key][kk].to(IF_device)

    for key in tqdm(val_grad_avg_dict[epoch].keys()):
        # for kk in val_grad_avg_dict[epoch][key]:
        val_grad_avg_dict[epoch][key] = val_grad_avg_dict[epoch][key].to(IF_device)

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [16]:
val_grad_avg_dict[0]['base_model.model.classifier.modules_to_save.default.weight'].device

device(type='cuda', index=3)

In [23]:
val_grad_avg_dict[0]['base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_B.default.weight'].device

device(type='cpu')

In [34]:
IF_dict_all = {}
for weight_name in tqdm(val_grad_avg_dict[0].keys()):
    _,IF = compute_hvp_and_IF(tr_grad_dict=tr_grad_dict[0],
                            val_grad_avg_dict=val_grad_avg_dict[0],
                            weight_name=weight_name,
                            device=IF_device)
    IF_dict_all[weight_name] = IF

  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/ipykernel_3177074/675338865.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hvp_iterative_dict[weight_name] = torch.tensor(val_grad_avg_dict_layer @ G_l_inv)


Time taken for HyperINF:  1.3718876838684082


/tmp/ipykernel_3177074/675338865.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hvp_iterative_dict[weight_name] = torch.tensor(val_grad_avg_dict_layer @ G_l_inv)


Time taken for HyperINF:  1.3316936492919922


...

Time taken for HyperINF:  1.4087378978729248


/tmp/ipykernel_3177074/675338865.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hvp_iterative_dict[weight_name] = torch.tensor(val_grad_avg_dict_layer @ G_l_inv)


Time taken for HyperINF:  1.4120762348175049


/tmp/ipykernel_3177074/675338865.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hvp_iterative_dict[weight_name] = torch.tensor(val_grad_avg_dict_layer @ G_l_inv)


Time taken for HyperINF:  1.4067926406860352
